# Activity completion
This code will complete the activities of a person for a day, given the first activity of the person.

In [15]:
import pandas as pd
import numpy as np
import dill
import time

In [16]:
df = pd.read_csv('first_activities.csv')
df = df[['id','arr_time','dep_time','purpose','employment','X_student','gender','education','driver_license','X_transit_pass']]
df['next_purpose'] = df['purpose']
df = pd.get_dummies(df, columns=['next_purpose'])
df

,id,arr_time,dep_time,purpose,employment,X_student,gender,education,driver_license,X_transit_pass,...,next_purpose_17,next_purpose_18,next_purpose_19,next_purpose_20,next_purpose_21,next_purpose_22,next_purpose_23,next_purpose_24,next_purpose_25,next_purpose_26
0,10319851,12,28,1,0,0,1,6,1,0,...,0,0,0,0,0,0,0,0,0,0
1,10319852,12,76,1,0,0,2,6,1,0,...,0,0,0,0,0,0,0,0,0,0
2,10320364,12,52,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,10320531,12,57,1,1,0,2,3,1,0,...,0,0,0,0,0,0,0,0,0,0
4,10320534,12,44,1,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11484,72116904,12,36,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
11485,72117981,12,35,1,1,0,2,5,1,0,...,0,0,0,0,0,0,0,0,0,0
11486,72118624,12,48,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
11487,72119121,12,38,1,0,0,2,3,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# ATM
with open('ATM.pkl', 'rb') as f:
    loaded_model_data = dill.load(f)
ATM = loaded_model_data['model']
group_probs = loaded_model_data['group_probs']
# ADM
with open('ADM.pkl', 'rb') as f:
    ADM = dill.load(f)

In [18]:
def generate_activity_sequence(person_data, ATM, ADM):
    activity_sequence = [person_data.copy()]  # Start with a copy of the person_data Series
    while True:
        current_activity = activity_sequence[-1].copy()  # Make a copy of the last Series in the list
        next_purpose = ATM(current_activity['dep_time'], current_activity['purpose'], current_activity['employment'], current_activity['X_student'])
        if next_purpose == 'None':
            break  # End the trip if the prediction is None
        current_activity[f'next_purpose_{int(next_purpose)}'] = 1
        current_activity['next_purpose_7'] = current_activity.get('next_purpose_7', 0)
        current_activity['next_purpose_9'] = current_activity.get('next_purpose_9', 0)

        current_activity['next_purpose_10'] = current_activity.get('next_purpose_10', 0)
        current_activity['next_purpose_16'] = current_activity.get('next_purpose_16', 0)

        current_features = pd.DataFrame(
            {
                'purpose_1': [current_activity['next_purpose_1']],
                'purpose_2': [current_activity['next_purpose_2']],
                'purpose_3': [current_activity['next_purpose_3']],
                'purpose_4': [current_activity['next_purpose_4']],
                'purpose_5': [current_activity['next_purpose_5']],
                'purpose_6': [current_activity['next_purpose_6']],
                'purpose_7': [current_activity['next_purpose_7']],
                'purpose_8': [current_activity['next_purpose_8']],
                'purpose_9': [current_activity['next_purpose_9']],
                'purpose_10': [current_activity['next_purpose_10']],
                'purpose_11': [current_activity['next_purpose_11']],
                'purpose_12': [current_activity['next_purpose_12']],
                'purpose_13': [current_activity['next_purpose_13']],
                'purpose_14': [current_activity['next_purpose_14']],
                'purpose_15': [current_activity['next_purpose_15']],
                'purpose_16': [current_activity['next_purpose_16']],
                'purpose_17': [current_activity['next_purpose_17']],
                'purpose_18': [current_activity['next_purpose_18']],
                'purpose_19': [current_activity['next_purpose_19']],
                'purpose_20': [current_activity['next_purpose_20']],
                'purpose_21': [current_activity['next_purpose_21']],
                'purpose_22': [current_activity['next_purpose_22']],
                'purpose_23': [current_activity['next_purpose_23']],
                'purpose_24': [current_activity['next_purpose_24']],
                'purpose_25': [current_activity['next_purpose_25']],
                'purpose_26': [current_activity['next_purpose_26']],
                'arr_time': [current_activity['dep_time']+6],
                'education': [current_activity['education']],
                'driver_license': [current_activity['driver_license']]
            }
        )        
        next_duration = ADM.predict(current_features)
        next_activity = create_next_activity(current_activity, next_purpose, next_duration)
        activity_sequence.append(next_activity)
    return pd.DataFrame(activity_sequence)  # Convert the list of Series to a DataFrame

def create_next_activity(current_activity, next_purpose, next_duration):
    next_activity = current_activity.copy()
    next_activity['purpose'] = next_purpose
    next_activity['arr_time'] = current_activity['dep_time'] + 2
    next_activity['dep_time'] = next_activity['arr_time'] + next_duration//1
    for i in range(1, 10):
        next_activity[f'next_purpose_{i}'] = 0
    return next_activity

In [19]:
from tqdm import tqdm

start = time.time()
activity_sequences = pd.DataFrame()  # Initialize an empty DataFrame
for _, person_data in tqdm(df.iterrows(), total=df.shape[0]):
    activity_sequence = generate_activity_sequence(person_data, ATM, ADM)
    activity_sequences = pd.concat([activity_sequences, activity_sequence], ignore_index=True)
end = time.time()
print(f"Execution Time: {end - start} seconds")

100%|████████████████████████████████████████████████████████████████████████████| 11489/11489 [03:40<00:00, 52.07it/s]

Execution Time: 220.6300654411316 seconds


In [20]:
# if the last activity of a person is home ('purpose' = 1) for the person or last activity's dep_time>323, define the last activity's dep_time as 323
activity_sequences= activity_sequences[['id','arr_time','dep_time','purpose']]
def modify_dep_time(df_group):
    if df_group['purpose'].iloc[-1] == 1 or df_group['dep_time'].iloc[-1] > 107:
        df_group['dep_time'].iloc[-1] = 107
    return df_group

activity_sequences = activity_sequences.groupby('id').apply(modify_dep_time)

In [21]:
activity_sequences.to_csv('activities_scheduler.csv',index = False)

In [14]:
activity_sequences

,id,arr_time,dep_time,purpose
0,10319851.0,12.0,32.0,1.0
1,10319851.0,34.0,63.0,2.0
2,10319851.0,65.0,77.0,25.0
3,10319851.0,79.0,107.0,1.0
4,10319852.0,12.0,37.0,1.0
...,...,...,...,...
41709,72119121.0,81.0,107.0,1.0
41710,72120271.0,26.0,34.0,1.0
41711,72120271.0,36.0,67.0,22.0
41712,72120271.0,69.0,75.0,18.0
